# Batch Activation Patching Experiments

This notebook allows you to experiment with batch activation extraction and aggregation. Instead of using activations from a single text, you can:

1. **Extract activations from multiple positive texts**
2. **Aggregate them using different methods** (mean, median, max, etc.)
3. **Use the aggregated activation for patching**
4. **Compare different aggregation strategies**

This approach may provide more robust and generalizable activation patterns for patching.

In [ ]:
import sys
import os
import torch
import json
import random
import numpy as np
from IPython.display import display, HTML

# Add TransformerLens to path
sys.path.append('/home/koalacrown/Desktop/Code/Projects/turnaround/turn_point/third_party/TransformerLens')

# Import our activation patcher
sys.path.append('/home/koalacrown/Desktop/Code/Projects/turnaround/turn_point/manual_activation_patching')
from activation_patcher import ActivationPatcher

print("Imports successful!")

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

## Choose Model and Load Dataset

In [ ]:
# Choose your model here - change this and re-run to experiment with different models
MODEL_NAME = "gpt2-small"  # Change to: gpt2-medium, EleutherAI/gpt-neo-125m, etc.

# Initialize the activation patcher
patcher = ActivationPatcher(MODEL_NAME)

# Load the positive patterns dataset
dataset_path = "/home/koalacrown/Desktop/Code/Projects/turnaround/turn_point/data/final/positive_patterns.jsonl"
patterns = patcher.load_dataset(dataset_path)

print(f"Loaded {len(patterns)} positive thought patterns")
print(f"Model info: {patcher.get_model_info()}")

## Explore Available Models

In [ ]:
# List all supported models
ActivationPatcher.list_supported_models()

## Prepare Batch Data

In [ ]:
# Select patterns by cognitive type or randomly
def filter_patterns_by_type(patterns, pattern_type=None, max_count=20):
    """Filter patterns by cognitive type or return random selection."""
    if pattern_type:
        filtered = [p for p in patterns if pattern_type.lower() in p.get('cognitive_pattern_type', '').lower()]
        print(f"Found {len(filtered)} patterns matching '{pattern_type}'")
    else:
        filtered = patterns.copy()
        random.shuffle(filtered)
        print(f"Using random selection from {len(filtered)} patterns")
    
    return filtered[:max_count]

# Available cognitive pattern types in the dataset:
pattern_types = set([p.get('cognitive_pattern_type', '') for p in patterns])
print("Available cognitive pattern types:")
for ptype in sorted(pattern_types):
    if ptype:
        count = len([p for p in patterns if p.get('cognitive_pattern_type', '') == ptype])
        print(f"  - {ptype} ({count} examples)")

In [ ]:
# Configuration for batch experiments
BATCH_SIZE = 15  # Number of texts to use for batch activation extraction
PATTERN_TYPE = None  # Set to specific type like "rumination" or None for random

# Select patterns for batch
selected_patterns = filter_patterns_by_type(patterns, PATTERN_TYPE, BATCH_SIZE)

# Extract texts
batch_texts = [p['positive_thought_pattern'] for p in selected_patterns]

print(f"\nSelected {len(batch_texts)} texts for batch processing:")
for i, text in enumerate(batch_texts[:5]):  # Show first 5
    print(f"{i+1}. {text[:100]}...")
if len(batch_texts) > 5:
    print(f"... and {len(batch_texts) - 5} more")

## Experiment 1: Different Aggregation Methods

In [ ]:
# Test different aggregation methods
aggregation_methods = ["mean", "median", "max", "random"]
corrupted_prompt = "I feel completely overwhelmed and stuck, unable to"
target_words = ["positive", "solutions", "growth", "hope", "progress"]

print(f"Testing aggregation methods with:")
print(f"- Batch size: {len(batch_texts)}")
print(f"- Corrupted prompt: {corrupted_prompt}")
print(f"- Target words: {target_words}")
print("\n" + "="*100)

aggregation_results = {}

for method in aggregation_methods:
    print(f"\n--- Testing {method.upper()} aggregation ---")
    
    try:
        predicted_token, generated_text = patcher.batch_patch_and_generate(
            clean_texts=batch_texts,
            corrupted_text=corrupted_prompt,
            layer_idx=-1,
            aggregation=method,
            target_words=target_words,
            num_placeholder_tokens=5,
            max_new_tokens=60
        )
        
        aggregation_results[method] = {
            'success': True,
            'predicted_token': predicted_token,
            'generated_text': generated_text
        }
        
        print(f"\n✓ SUCCESS with {method} aggregation:")
        print(f"Generated: {generated_text}")
        
    except Exception as e:
        print(f"✗ Error with {method} aggregation: {e}")
        aggregation_results[method] = {
            'success': False,
            'error': str(e),
            'generated_text': None
        }
    
    print("-" * 80)

# Memory cleanup
if torch.cuda.is_available():
    torch.cuda.empty_cache()

## Experiment 2: Batch Size Comparison

In [ ]:
# Test different batch sizes
batch_sizes = [1, 3, 5, 10, 15]
corrupted_prompt = "My thoughts are spiraling and I can't seem to"
aggregation_method = "mean"

print(f"Testing batch sizes with {aggregation_method} aggregation")
print(f"Corrupted prompt: {corrupted_prompt}")
print("\n" + "="*100)

batch_size_results = {}

for batch_size in batch_sizes:
    if batch_size > len(batch_texts):
        print(f"Skipping batch size {batch_size} (not enough texts available)")
        continue
        
    print(f"\n--- Testing batch size: {batch_size} ---")
    
    subset_texts = batch_texts[:batch_size]
    
    try:
        predicted_token, generated_text = patcher.batch_patch_and_generate(
            clean_texts=subset_texts,
            corrupted_text=corrupted_prompt,
            layer_idx=-1,
            aggregation=aggregation_method,
            target_words=None,
            num_placeholder_tokens=5,
            max_new_tokens=55
        )
        
        batch_size_results[batch_size] = {
            'success': True,
            'generated_text': generated_text,
            'predicted_token': predicted_token
        }
        
        print(f"\n✓ SUCCESS with batch size {batch_size}:")
        print(f"Generated: {generated_text}")
        
    except Exception as e:
        print(f"✗ Error with batch size {batch_size}: {e}")
        batch_size_results[batch_size] = {
            'success': False,
            'error': str(e),
            'generated_text': None
        }
    
    print("-" * 80)

if torch.cuda.is_available():
    torch.cuda.empty_cache()

## Experiment 3: Custom Batch Configuration

In [ ]:
# Custom configuration - modify these variables for your experiments
CUSTOM_BATCH_TEXTS = [
    "I'm learning to acknowledge my feelings and take things one step at a time.",
    "I choose to focus on solutions rather than dwelling on problems.",
    "I'm practicing self-compassion and recognizing my growth.",
    "I can handle challenges by breaking them into manageable pieces.",
    "I'm building resilience and finding healthy ways to cope.",
    "I trust in my ability to navigate difficult situations.",
    "I'm grateful for the support I have and the progress I've made.",
    "I choose hope and believe that positive change is possible."
]

CUSTOM_CORRUPTED_TEXT = "I don't know how to deal with this situation and feel"
CUSTOM_AGGREGATION = "mean"
CUSTOM_TARGET_WORDS = ["acknowledge", "focus", "solutions", "growth", "hope"]
CUSTOM_LAYER = -1
CUSTOM_PLACEHOLDERS = 5
CUSTOM_MAX_TOKENS = 70

print("CUSTOM BATCH EXPERIMENT:")
print("="*100)
print(f"Batch texts ({len(CUSTOM_BATCH_TEXTS)} total):")
for i, text in enumerate(CUSTOM_BATCH_TEXTS[:3]):
    print(f"  {i+1}. {text}")
if len(CUSTOM_BATCH_TEXTS) > 3:
    print(f"  ... and {len(CUSTOM_BATCH_TEXTS) - 3} more")

print(f"\nCorrupted text: {CUSTOM_CORRUPTED_TEXT}")
print(f"Aggregation: {CUSTOM_AGGREGATION}")
print(f"Target words: {CUSTOM_TARGET_WORDS}")
print(f"Layer: {CUSTOM_LAYER}")
print(f"Placeholders: {CUSTOM_PLACEHOLDERS}")
print(f"Max tokens: {CUSTOM_MAX_TOKENS}")

try:
    predicted_token, generated_text = patcher.batch_patch_and_generate(
        clean_texts=CUSTOM_BATCH_TEXTS,
        corrupted_text=CUSTOM_CORRUPTED_TEXT,
        layer_idx=CUSTOM_LAYER,
        aggregation=CUSTOM_AGGREGATION,
        target_words=CUSTOM_TARGET_WORDS,
        num_placeholder_tokens=CUSTOM_PLACEHOLDERS,
        max_new_tokens=CUSTOM_MAX_TOKENS
    )
    
    print("\n" + "="*50)
    print("CUSTOM EXPERIMENT RESULT:")
    print("="*50)
    print(generated_text)
    print("="*50)
    
except Exception as e:
    print(f"\n✗ Error in custom experiment: {e}")

## Experiment 4: Baseline Comparison

In [ ]:
# Comparison experiment
comparison_prompt = "I'm overwhelmed by everything and don't see a way to"
batch_for_comparison = batch_texts[:10]
single_text_for_comparison = batch_texts[0]

print("BASELINE COMPARISON:")
print("="*100)
print(f"Test prompt: {comparison_prompt}")
print(f"Batch size: {len(batch_for_comparison)}")

comparison_results = {}

# 1. No patching (baseline)
print("\n--- 1. NO PATCHING (Baseline) ---")
try:
    tokens = patcher.model.to_tokens(comparison_prompt)
    generated_tokens = patcher.model.generate(
        tokens,
        max_new_tokens=60,
        temperature=0.7,
        do_sample=True
    )
    baseline_text = patcher.model.to_string(generated_tokens[0])
    comparison_results['no_patching'] = baseline_text
    print(f"Generated: {baseline_text}")
except Exception as e:
    print(f"Error: {e}")
    comparison_results['no_patching'] = f"Error: {e}"

# 2. Single text patching
print("\n--- 2. SINGLE TEXT PATCHING ---")
try:
    target_words = patcher._extract_key_words(single_text_for_comparison)
    predicted_token, single_patch_text = patcher.patch_and_generate(
        clean_text=single_text_for_comparison,
        corrupted_text=comparison_prompt,
        target_words=target_words,
        max_new_tokens=60
    )
    comparison_results['single_patching'] = single_patch_text
    print(f"Generated: {single_patch_text}")
except Exception as e:
    print(f"Error: {e}")
    comparison_results['single_patching'] = f"Error: {e}"

# 3. Batch patching
print("\n--- 3. BATCH PATCHING ---")
try:
    predicted_token, batch_patch_text = patcher.batch_patch_and_generate(
        clean_texts=batch_for_comparison,
        corrupted_text=comparison_prompt,
        aggregation="mean",
        max_new_tokens=60
    )
    comparison_results['batch_patching'] = batch_patch_text
    print(f"Generated: {batch_patch_text}")
except Exception as e:
    print(f"Error: {e}")
    comparison_results['batch_patching'] = f"Error: {e}"

print("\n" + "="*100)
print("COMPARISON SUMMARY:")
print("="*100)
for method, result in comparison_results.items():
    print(f"\n{method.upper().replace('_', ' ')}:")
    print(result)
    print("-" * 60)

## Results Summary

In [ ]:
print("BATCH ACTIVATION PATCHING EXPERIMENT SUMMARY")
print("="*100)

print(f"\nModel Used: {patcher.model_name}")
print(f"Model Info: {patcher.get_model_info()}")

print(f"\nDataset: {len(patterns)} total patterns")
print(f"Batch Size Tested: {len(batch_texts)} texts")

experiments = [
    ("Aggregation Methods", globals().get('aggregation_results', {})),
    ("Batch Sizes", globals().get('batch_size_results', {})),
    ("Baseline Comparison", globals().get('comparison_results', {}))
]

for exp_name, results in experiments:
    print(f"\n{exp_name}:")
    if results:
        success_count = sum(1 for r in results.values() if isinstance(r, dict) and r.get('success', False))
        total_count = len(results)
        print(f"  - Completed: {success_count}/{total_count} configurations")
        
        if success_count > 0:
            successful_configs = [k for k, v in results.items() if isinstance(v, dict) and v.get('success', False)]
            if not successful_configs:
                successful_configs = [k for k in results.keys() if not str(results[k]).startswith('Error')]
            print(f"  - Successful configurations: {successful_configs}")
    else:
        print(f"  - Not run in this session")

print("\nKey Findings:")
print("- Batch activation patching allows for more robust activation patterns")
print("- Different aggregation methods can produce varied results")
print("- Batch size affects the quality and consistency of patching")
print("- Comparison with baseline shows the effect of activation patching")

print("\nNext Steps:")
print("- Try different models by changing MODEL_NAME and re-running")
print("- Experiment with different cognitive pattern types")
print("- Test with longer generation sequences")
print("- Try different layers for patching")